__Zarr Development Notebooks__</br>
<img src="http://static1.squarespace.com/static/530979d9e4b04bff4a3aadf5/t/5446c34ce4b0e5a2c7ff2614/1413923661373/DewberryLogo_RGB.png?format=1500w" width="40%" align='right'/>
__Probability Risk: Post Processor Group__</br>
PYTHON 3.6</br>
Overview: This notebook is to analyze how to store WSE tifs from multiple models into one cube of data.</br>
Updated 2018-10-15</br>
by Stephen Duncan: sduncan@dewberry.com <br/>

*Use Environment with boto3, zarr, & s3fs*

# Read Zarr File Tutorial

 - [Zarr Tutorial & Documentation](https://zarr.readthedocs.io/en/stable/tutorial.html)

# Python Librarys Needed

In [1]:
import os, sys, time, glob, s3fs, zarr, json, boto3
import numpy as np
import pandas as pd
from io import BytesIO
from osgeo import gdal, osr
from matplotlib import pyplot as plt
from IPython.display import display, Markdown, Latex
%matplotlib inline

import zarr_aws #Script with functions

# Define and read in zarr group

In [2]:
bucket2 = 'probmodelingrepository'
prefix2 = 'zarr'
zarr_file_name = 'demo.zarr'
zarr_file = f's3://{bucket2}/{prefix2}/{zarr_file_name}'
zarr_path = 'WSE'

zarr_store = s3fs.S3Map(root=zarr_file, s3=s3fs.S3FileSystem(), check=False)
z = zarr.open_group(store=zarr_store, path=zarr_path, mode='r')

# From part 1, we can see the included groups

We do this using the `zarrgroup.attrs.keys()` function

In [3]:
z.attrs.keys()

dict_keys(['WSE_Pluvial_E1181', 'WSE_Pluvial_E1182'])

## We can use these group keys to get the metadata from each Tif

In [7]:
z.attrs['WSE_Pluvial_E1181'].keys()

dict_keys(['Cell Size', 'Extents', 'Full GetStatistics', 'Projection', 'Unit Type', 'XYSize'])

# With these group keys we can use them to search chunk data within each group.

In [8]:
zarr_path = 'WSE'
for key in z.attrs.keys():
    z1 = zarr.open_group(store=zarr_store, path=f'{zarr_path}/{key}', mode='r')
    print(f'{key}:', list(z1.attrs.keys()))

WSE_Pluvial_E1181: ['r0_c0']
WSE_Pluvial_E1182: ['r0_c0']


# Read can now read in the different arrays

In [5]:
zarrays = []
zarr_path = 'Bool'
for key in z.attrs.keys():
    z1 = zarr.open_group(store=zarr_store, path=f'{zarr_path}/{key}', mode='r')
    for ckey in z1.attrs.keys():
        print(f'key: {key}, ckey: {ckey}')
        zrry = z1[ckey]
        zarrays.append(zrry)

key: WSE_Pluvial_E1181, ckey: r0_c0
key: WSE_Pluvial_E1182, ckey: r0_c0


In [6]:
npzarr = np.array(zarrays[0])
npzarr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)